In [ ]:
import torch
x = torch.randn(100,4)
x = x.reshape(10,-1,4)
print(x.shape)
probabilities = torch.softmax(x, dim=1).view(-1, 4)
# 计算每个样本概率总和并获取最大概率的位置索引
print(probabilities.shape)
sum_probs = torch.sum(probabilities, dim=0)
print(sum_probs)
max_prob_idx = torch.argmax(sum_probs, dim=0)
max_prob_idx.item()

In [ ]:
import torch
from torch.nn import functional
a = torch.randn(10,2)

lin = torch.nn.Linear(2,5,False)
for param in lin.parameters():
    print(param)
# print(lin.named_modules())
print(lin(a))
# print(torch.mul(a,(lin.weight).T))
functional.linear(a,lin.weight)

In [ ]:
import torch
from torch.distributions import RelaxedBernoulli

# 参数设置
temperature = 0.5
logits = torch.tensor([1.0, 2.0, -1.0])  # 输入的logits

# 将logits转化为RelaxedBernoulli分布对象
distribution = RelaxedBernoulli(temperature=temperature, logits=logits)

# 采样一个样本
sample = distribution.rsample()

print(sample)  # 打印采样的样本

# 如果想获取离散的样本，可以使用一个适当的阈值进行比较
discrete_sample = sample.ge(0.5).float()

print(discrete_sample)  # 打印转化为离散形式的样本


In [ ]:
sample = distribution.rsample()
sample

In [2]:
import re
f = open('account.txt')
accs = f.readlines()


res = []
res2 = []
res3 = []
for acc in accs:
    if acc.find('INSERT') >=0:
        continue
    acc = eval(acc[:-2])
    user = acc[0]
    passord = acc[1]
    role = acc[3]
    if role.find('二级密码')>=0:
        continue
    pattern = r'\【(.*?)\】'  # 匹配【】之间的字符

    format = f'{user}----{passord}----{7}----{role}'
    result = re.findall(pattern, role)
    if result:
        l = ['莫娜','七七','迪卢克','琴','刻晴','提纳里','迪希雅']
        target_string = result[0]
        array = target_string.split('|')
        count = 0
        for r in array:
            if r not in l:
                count = count + 1
        if count >= 3:
            res.append(format)
        else:
            res2.append(format)
            for r in array:
                for role in ['雷电将军*','胡桃*','枫原万叶*','夜兰*','纳西妲*','神里绫华*']:
                    if r.find(role) >= 0:
                        res3.append(format)
                        break
           
        print(count)
    # print(role)
# print(res)
f = open('1.txt','w')
for p in res:
    f.write(p + '\n')

f = open('2.txt','w')
for p in res2:
    f.write(p + '\n')    

f = open('3.txt','w')
for p in res3:
    f.write(p + '\n')   

2
1
3
2
4
3
1
3
2
2
2
4
1
2
3
1
3
1
2
2
3
2
2
3
2
1
2
3
1
5
1
1
2
1
1
2
3
3
3
3
1
1
2
2
2
2
2
2
2
1
2
3
2
2
1
2
1
2
2
3
3
2
1
3
3
4
3
2
3
3
2
1
1
2
1
2
2
2
2
2
3
3
3
1
1
1
2
2
3
1
3
1
5
2
2
2
2
1
3
2
3
2
2
1
2
1
2
2
3
1
2
1
1
1
3
2
1
2
3
2
3
1
1
4
1
2
1
3
2
3
4
1
2
1
2
2
1
3
1
2
1
1
2
2
1
2
2
2
4
1
4
1
1
3
3
2
2
2
2
2
2
1
2
2
1
3
1
1
3
3
4
2
1
2
3
2
3
1
2
2
1
4
3
3
2
1
3
2
2
1
1
1
1
3
2
2
2
2
2
2
1
2
1
3
1
3
3
1
2
3
1
1
3
2
2
2
2
3
3
2
3
3
1
2
2
1
1
2
2
2
1
2
4
3
5
2
1
1
1
2
1
3
3
1
4
2
1
3
1
2
1
1
3
4
2
4
1
2
2
2
2
2
2
1
3
2
2
2
3
3
2
1
1
1
2
1
3
2
2
3
3
3
2
3
3
2
2
1
3
3
2
3
2
3
2
2
1
2
4
1
3
2
2
2
2
2
2
1
2
2
2
2
2
3
2
2
2
1
2
1
2
2
3
2
2
2
1
2
2
2
2
1
2
1
1
3
1
2
3
2
2
2
3
2
2
1
2
2
2
2
2
2
2
2
1
2
1
2
1
1
1
4
2
1
2
2
1
3
3
2
2
3
2
2
2
1
3
5
3
1
1
1
1
3
2
3
2
2
2
2
2
3
1
2
2
2
2
5
1
1
3
1
1
1
2
2
2
2
2
2
2
4
3
3
2
1
2
2
3
2
2
2
2
1
2
3
1
3
1
2
3
3
1
3
2
2
2
4
2
1
3
3
2
1
2
1
1
1
1
1
1
2
2
1
2
3
2
1
2
2
2
3
1
2
2
2
3
1
3
2
2
3
2
3
2
1
2
2
1
2
1
3
3
3
3
1
1
2
2
3
2
1
1
1
1
2
2
3
3
2


In [1]:
f = open('需冲.txt')

f2 = open('充值.txt')


f3 = open('res.txt','w')

l1 = f.readlines()
l2 = f2.readlines()
res = []
for a in l1:
    found = 0
    for b in l2:
        if a.split('----')[0]==b.split('----')[0]:
            found = 1
            break
    if not found:
        f3.write(a)

In [2]:
f = open('需冲.txt')
f2 = open('天.txt')
f3 = open('res.txt','w')
l1 = f.readlines()
l2 = f2.readlines()
res = []
for a in l1:
    found = 0
    for b in l2:
        if a.split('----')[0]==b.split('----')[0]:
            found = 1
            break
    if not found:
        f3.write(a)
# for a in l2:
#     found = 0
#     for b in l1:
#         if a.split('----')[0]==b.split('----')[0]:
#             found = 1
#             break
#     if not found:
#         f3.write(a)